# What is ROOT ? #

Program package for data acquisition, simulation and data analysis which is being developed at CERN (European Center for Particle Physics, Geneva).

*  based on C++
*  object oriented

ROOT provides a variety of objects/functions

* Data I/O
* and data management
* simulation
* analysis
* statistics
* visualization

ROOT is a central part of the reconstruction and analysis software of all current experiments in particle physics and is also used a lot in other areas.

ROOT can be used both as a tool-kit in larger, standalone programs and for interactive data analysis and visualization.

We limit ourselves to the latter which is just a small part of the functionality:

* interactive work optionally using the C++ interpreter (CINT or CLANG)
* or Python (pyROOT)
* Execute directly from shell/command line
* or via Jupyter Notebooks


## Working with ROOT -- python

For interactive work jupyter notebook is rather convenient, both for Python and C++ interface.

ROOT home page: https://root.cern/

Detailed ROOT tutorial: https://www.nevis.columbia.edu/~seligman/root-class/

ROOT is not default package for python, it requires special installation and setup: 

Select environment Environment `root/6.24.02 ` when you start on *jupyterhub* on *https://jupyter.physik.uni-muenchen.de* 

Then do `import ROOT` to get access to ROOT classes and functions

In [1]:
import ROOT
# enable interactive plotting features
%jsroot on 

Welcome to JupyROOT 6.24/02


### Python/ROOT as pocket calculator

In [2]:
sin(0.5) # does not work -- sin not known

NameError: name 'sin' is not defined

In [3]:
import math # need module math
math.sin(0.5)

0.479425538604203

In [4]:
math.atan(1)*4

3.141592653589793

In [5]:
# or equivalent using ROOT.TMath funcs:
ROOT.TMath.Sin(0.5)

0.479425538604203

In [6]:
ROOT.TMath.ATan(1)*4

3.141592653589793

In [ ]:
# check what functions are available
help(ROOT.TMath)

### ROOT classes
ROOT provides users with a huge class library for functions, histograms, random numbers, statistics, I/O , etc.

In practice, working with ROOT means that you create objects of the respective ROOT classes and then call methods of these objects.

Simple examples:

#### 1D Histogram ####

In [8]:
import ROOT

# book histo,
# arguments: tag, title, N-channels, xlow, x-high
myhist1 = ROOT.TH1F("h1","Gauss Random Numbers",100,-3.,3.)
# random generator object
rng = ROOT.TRandom()
# fill histo in loop
for i in range(100000):
    xrnd = rng.Gaus() # Gaussian distributed Random number
    myhist1.Fill( xrnd ) # Fill random number in histogram

# need drawing canvas for jupyter    
myc = ROOT.TCanvas("myc","myc",10,10,500,300);     
myhist1.Draw() # Draw Histogramm
myc.Draw() # draw canvas

#### 2D Histogram ####

In [11]:
# 2D hist example
myhist2 = ROOT.TH2F("h2","2D Gauss Random Numbers",100,-5.,5.,100,-5.,-5.)
# random generator object
rng = ROOT.TRandom()
# fill histo in loop
for i in range(100000):
    xrnd = rng.Gaus() # Gaussian distributed Random number
    yrnd = rng.Gaus()
    myhist2.Fill( xrnd, xrnd+yrnd ) # Fill random number in histogram

# in Jupyter we need to create explict Canvas ...
c = ROOT.TCanvas("c","myCanvas",50,50,600,600)
#myhist2.Draw() # Draw Histogramm as scatter plot
myhist2.Draw('lego2') # lego plot
# ... and call Draw for Canvas
c.Draw()  

Warning in <TROOT::Append>: Replacing existing TH1: h2 (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


many options how to draw 2D histogram, see more in https://root.cern/doc/master/draw2dopt_8C.html


### Drawing functions ###

In [12]:
#myc.Clear()
myc = ROOT.TCanvas("myc","myc",10,10,800,600);     
myc.Divide(2,2) # make 2x2 subpads
myc.cd(1)
f1 = ROOT.TF1("f1","sin(x)", 0, 10 )
f1.Draw()
myc.cd(2) # next pad
f2 = ROOT.TF1("f2","cos(x)", 0, 10 )
f2.SetLineColor(1);
f2.Draw()
f1.Draw("same")
mypad=myc.cd(3)
mypad.SetLogy() # log y scale
mypad.SetGrid() # grid lines
f4 = ROOT.TF1("f4","abs(f1)*exp(x)", 0, 10 )
f4.Draw()
myc.Draw()





Warning in <TCanvas::Constructor>: Deleting canvas with same name: myc


#### x-y Plot with error bars ####
Scientific data, in particular in physics, often has errors associated to the y-coordinate (sometimes also to x)

In [13]:
import numpy as np
x = np.linspace(0, 10, 50)
y = np.sin(x) + 0.5*np.random.randn(50)
ey = np.ones(len(x))*0.7
tg = ROOT.TGraphErrors(len(x), x, y, 0, ey) # graph with error bars
tg.SetMarkerStyle(20);

c = ROOT.TCanvas("c","myCanvas",500,400)
tg.Draw('AP')
c.Draw()  


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c





Let's define short helper function to combine canvas and object drawing:

In [15]:
def nbdraw( myobj, myopts="", mysize=(500,400) ):
    "helper function for drawing in notebook"
    c = ROOT.TCanvas("myc","myCanvas",*mysize);
    myobj.Draw(myopts)
    c.Draw()  
    return c

    

In [16]:
nbdraw(tg, 'AP')

Warning in <TCanvas::Constructor>: Deleting canvas with same name: myc


---
#### Simple fitting examples ####

1. Gaus fit to 1d histogram

In [18]:
fr = myhist1.Fit("gaus")
fit = myhist1.GetFunction("gaus")
mean = fit.GetParameter(1)
emean = fit.GetParError(1)
print ( f" {mean = :.5f}  +-  {emean:.5f}")
# need drawing canvas for jupyter    
#myhist1
ROOT.gStyle.SetOptFit(1) # global setting to have fit results in stat box
nbdraw(myhist1)

 mean = 0.00059  +-  0.00321


 FCN=81.3965 FROM MIGRAD    STATUS=CONVERGED      51 CALLS          52 TOTAL
                     EDM=7.38425e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.39360e+03   9.42127e+00   3.36083e-02   8.85273e-05
   2  Mean         5.89379e-04   3.20833e-03   1.42221e-05   1.25903e-01
   3  Sigma        9.99324e-01   2.39819e-03   2.87404e-06   1.76457e+00


Warning in <TCanvas::Constructor>: Deleting canvas with same name: myc


---
2nd fit example: polynom fit to x-y points in TGraphErrors 

*try different polynomial degrees*

In [21]:
tgf = tg.Fit("pol5") # fit polynomial 4th order
nbdraw(tg,'AP')




****************************************
Minimizer is Linear / Migrad
Chi2                      =      25.7334
NDf                       =           44
p0                        =    -0.683553   +/-   0.503624    
p1                        =      3.02648   +/-   1.0535      
p2                        =     -1.71849   +/-   0.669633    
p3                        =     0.337974   +/-   0.171703    
p4                        =   -0.0260231   +/-   0.0190031   
p5                        =  0.000636925   +/-   0.000756196 


Warning in <TCanvas::Constructor>: Deleting canvas with same name: myc


#### Exercises - 1 ####
ATLAS drift tube postions with ROOT

* Read the data from [rohr1.dat](http://www-static.etp.physik.uni-muenchen.de/kurs/comp20/uebungen/source/rohr1.dat) into ROOT

* **C++ IO**
```
ifstream data_file;
data_file.open("pipe1.dat");
doublex;
// book histogram ...
while ( data_file >>x ) // reads next value into x
{ // Fill histo ...
}
```
* **Python IO**
```
import numpy as np
data = np.loadtxt('rohr1.dat') # read all data in numpy array
# book histo ....
for x in data: # loop over data
  # fill histo
```
Create a histogram and fill in the values. ( Sample solution: .C , .py )

* And the same for [rohr2.dat](http://www-static.etp.physik.uni-muenchen.de/kurs/comp20/uebungen/source/rohr2.dat)  in a 2-dim histogram (*scatter-plot*).

```
TH2F h2("h","mytitle",nx,xlow,xhigh,ny,ylow,yhigh);
...
h2.Fill(x,y)
```
* **C++ IO for x,y**
```
...
while ( data_file >>x >>y) // reads next value pair into x and y
```
* **Python IO for x,y**
```
...
data2 = np.loadtxt('rohr2.dat') # read all data in numpy array
...
for x,y in data2: # loop over data2 entries 
```


---

### Simple data analysis ###

ROOT enables the efficient and fast analysis of very large amounts of data. 
In principle of course data can be saved in simple ASCII or .csv  formats and read in and processed with normal python or C/C++ commands. However, this is not very efficient, both in terms of space usage as well as fast I/O.
Using ROOT tuples is much more efficient, this way the data and its properties or variables are structured in the so-called tree format (trees).

Root trees are optimized for storing and efficiently processing particle physics event data .

Typically, events that are recorded in the detector are stored and processed in very different ways.

* **Raw data** contain all the detailed information of the sub-detectors, eg the drift times of each tube in the myinspectrometer that registered a signal. This requires a complex, deep tree structure:
   * ATLAS -> muon spectrometer ->  chamber ID ->  tube ID -> drift time 



* **reconstructed data:** tracks in the tracking detectors, blocks or clusters in the calorimeters, reconstructed decay vertices, .... 
   * Still requires complex tree structure


* **Summary data:** reconstructed abstract objects: jets, leptons, photons, missing momentum vector, ... 
   * still nested data: can be 1, 2, ... n jets, leptons, ....


* **Global summary:** number of tracks or jets, energy in calorimeter, ... 
   * Flat table layout, fixed number of parameters (=columns) per event is sufficient for easy characterization.


HEP data analysis at the final stage usually use the latter format...


In [22]:
import ROOT

f=ROOT.TFile.Open("http://www.etp.physik.uni-muenchen.de/kurs/comp10/uebungen/Z0-Versuch/data/ntz0mhmc.root")
mytree = ROOT.gROOT.FindObject("h5000") # get access to tree

In [23]:
mytree.Print()

******************************************************************************
*Tree    :h5000     : EVENT                                                  *
*Entries :   100000 : Total =         6819388 bytes  File  Size =    4373101 *
*        :          : Tree compression factor =   1.56                       *
******************************************************************************
*Br    0 :Run       : Float_t                                                *
*Entries :   100000 : Total  Size=     401083 bytes  File Size  =       2520 *
*Baskets :        6 : Basket Size=      64000 bytes  Compression= 152.38     *
*............................................................................*
*Br    1 :Event     : Float_t                                                *
*Entries :   100000 : Total  Size=     401107 bytes  File Size  =     133522 *
*Baskets :        6 : Basket Size=      64000 bytes  Compression=   2.88     *
*...................................................

In [29]:
c = ROOT.TCanvas("c","myCanvas",500,400)
h1 = ROOT.TH1F("nce","N Tracks + N Ecal", 81, 0, 100)
mytree.Draw("Ncharged+N_ecal >> nce")   # fuellt 1D Histogramm mit Variable Ncharged (automatische Histo Erzeugung)
#mytree.Draw("N_ecal:Ncharged")   # fuellt 2D Histogramm mit Variable Ncharged vs N_ecal
#h1 = ROOT.TH1F("nc","N Tracks", 50, 0, 50) #  Buche 1d Histo
#mytree.Draw("Ncharged>>nc")  # fuellt 1D Histogramm mit Variable Ncharged in gebuchtes Histo
#mytree.Draw("Ncharged>>nc","E_ecal>10") #  fuellt 1D Histogramm mit Variable Ncharged wenn Cut erfuellt ist
c.Draw()  

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Warning in <TWebFile::Append>: Replacing existing TH1: nce (Potential memory leak).


#### Exercises - 2 ####

* Plot the distributions (1D histograms) of important quantities like Ncharged, E_ecal, Pcharged for the datasets with simulated events: ntz0mhmc.root (quark/hadron decays), ntz0eemc.root (electron/positron), ntz0mmmc.root (muon), ntz0ttmc .root (tau)

* Example notebook for processing the files/trees and creating/plotting the histograms: PyROOT-Z0-Ex1.ipynb

* Make 2D histograms for the three combinations of Ncharged, E_ecal, Pcharged for both the simulated datasets and the detector data (ntz0e4.root).

